In [1]:
import os

import fire
from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client.types.agent_create_params import AgentConfig
from termcolor import colored

base_url = "http://localhost:8321"

client = LlamaStackClient(
    # base_url=f"http://{host}:{port}"
    base_url=base_url
)

available_shields = [shield.identifier for shield in client.shields.list()]
if not available_shields:
    print(colored("No available shields. Disabling safety.", "yellow"))
else:
    print(f"Available shields found: {available_shields}")

available_models = [
    model.identifier for model in client.models.list() if model.model_type == "llm"
]
if not available_models:
    print(colored("No available models. Exiting.", "red"))
    # return
else:
    selected_model = available_models[0]
    # selected_model="Qwen/Qwen3-14B"
    print(f"Using model: {selected_model}")

agent_config = AgentConfig(
    model=selected_model,
    instructions="You are a helpful assistant",
    sampling_params={
         "max_tokens":3096,"strategy": {"type": "top_p", "temperature": 1.0, "top_p": 0.9},
    },
    toolgroups=(
        [
            "mcp::crm"
        ]
    ),
    tool_choice="auto",
    input_shields=available_shields if available_shields else [],
    output_shields=available_shields if available_shields else [],
    enable_session_persistence=False
)
agent = Agent(client, agent_config)

user_prompts = [
    """"Get a list of active opportinities"""
]

session_id = agent.create_session("another-session")

for prompt in user_prompts:
    response = agent.create_turn(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        session_id=session_id,
    )

    for log in EventLogger().log(response):
        log.print()

`agent_config` is deprecated. Use inlined parameters instead.


No available shields. Disabling safety.
Using model: Qwen/Qwen3-14B
inference> <think>
Okay, the user is asking for a list of active opportunities. Let me check the available tools. There's a function called getOpportunities that retrieves active opportunities from the CRM. The parameters are empty, so I don't need any arguments. I should call that function. No need to use getSupportCases here since the query is about opportunities. Just make sure to structure the tool call correctly in JSON within the XML tags.
</think>

<tool_call>
{"name": "getOpportunities", "arguments": {}}
</tool_call>
